# Phase Problem in Electron Microscopy


> Georgios Varnavides | 07/16/2024  
> National Center for Electron Microscopy, Molecular Foundry, Berkeley Lab

In [1]:
%matplotlib widget

import tcbf
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display
import ipywidgets

# from ipywidgets import HBox, VBox, FloatSlider, FloatLogSlider, Checkbox

In [2]:
file_name = "apoF-ice-embedded-potential-binned.npy"
binned_volume_zxy = np.load("data/"+file_name)

In [3]:
style = {
    'description_width': 'initial',
}

defocus_slider = ipywidgets.FloatSlider(
    value = 0, min = -2, max = 2, 
    step = 0.05,
    description = r"defocus [$\mu$m]",
    style = style,
)


electrons_per_area_slider = ipywidgets.FloatLogSlider(
    value=1000,
    base=10,
    min=1, # min exponent of base
    max=3, # max exponent of base
    step=0.05, # exponent step
    description = r"electron dose [e/A$^2$]",
    style = style,
)

show_zernike_switch = ipywidgets.Checkbox(
    value=False,
    description="Zernike phase-plate",
    style=style,
)

def toggle_zernike(change):
    show_zernike = change['new']
    axs[1].axis("on" if show_zernike else "off")
    fig.canvas.draw_idle()
    return None

show_zernike_switch.observe(toggle_zernike,names='value')

In [4]:
# constants
semiangle = 4  # mrad
wavelength = 0.0197  # A (300kV)
sigma = 0.00065  # 1/V (300kV)
rolloff = 0.125  # mrad

In [5]:
# PotentialArray

pixel_size = 2 / 3
bin_factor_xy = 2
bin_factor_z = 6

potential = tcbf.PotentialArray(
    binned_volume_zxy,
    slice_thickness=pixel_size * bin_factor_z,
    sampling=(pixel_size * bin_factor_xy, pixel_size * bin_factor_xy),
)

potential.slice_thickness += 1e4 * defocus_slider.value / binned_volume_zxy.shape[0]

In [6]:
# Tilted Plane Wave
tilted_plane_wave = tcbf.Waves(
    array=np.ones(potential.gpts, dtype=np.complex64),
    sampling=potential.sampling,
    wavelength=wavelength,
    sigma=sigma,
    tilt=(0, 0),
)

In [7]:
# CTF
ctf = tcbf.CTF(
    semiangle_cutoff=semiangle,
    rolloff=rolloff,
)

In [8]:
# Angles
alpha, phi = tilted_plane_wave.get_scattering_angles()
bright_field_disk = np.fft.fftshift(ctf.evaluate_aperture(alpha, phi))

In [9]:
# Exit Waves

exit_wave = tilted_plane_wave.multislice(potential)
exit_wave = np.random.poisson(
    (
        np.abs(exit_wave) ** 2
        * np.prod(potential.sampling)
        * electrons_per_area_slider.value
    ).clip(0)
)
# exit_wave_zernike = np.fft.fft2(exit_wave)
# if show_zernike:
#     zernike_kernel = np.zeros_like(np.abs(exit_wave_zernike))
#     zernike_kernel[0, 0] = np.pi / 2
#     zernike_kernel = np.exp(1j * zernike_kernel)
#     exit_wave_zernike = np.fft.ifft2(exit_wave_zernike * zernike_kernel)
#     exit_wave_zernike = np.random.poisson(
#         (
#             np.abs(exit_wave_zernike) ** 2
#             * np.prod(potential.sampling)
#             * electrons_per_area
#         ).clip(0)
#     )



In [10]:
# Figures

with plt.ioff():
    dpi = 72
    fig, axs = plt.subplots(1,2, figsize=(675/dpi, 325/dpi), dpi=dpi)

tcbf.show(
    exit_wave,
    ticks=False,
    figax=(fig, axs[0]),
    cbar=False,
)

axs[0].set_title(
    "CTEM image intensity",
    fontsize=12,
)

tcbf.add_scalebar(
    axs[0],
    color="black",
    sampling=pixel_size * bin_factor_xy / 10,
    length=30,
    units="nm",
)
axs[1].axis("off")

    # tcbf.show(
    #     exit_wave_zernike,
    #     ticks=False,
    #     figax=(fig, axs[1]),
    #     cbar=False,
    # )

    # axs[1].set_title(
    #     "Zernike phase-plate intensity",
    #     fontsize=12,
    # )

    # tcbf.add_scalebar(
    #     axs[1],
    #     color="white",
    #     sampling=pixel_size * bin_factor_xy / 10,
    #     length=30,
    #     units="nm",
    # )


(np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0))

In [11]:
display(
    ipywidgets.VBox([
        fig.canvas,
        show_zernike_switch
    ])
)